In [1]:
import findspark
findspark.init("/usr/local/spark")
import pyspark
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)


In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import StopWordsRemover

path = "/opt/spark/data/mllib/attempt.txt"
italian_stop_words = ['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl', 'alla', 'alle', 'con', 'col', 'coi', 'da', 
                      'dal', 'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle', 'di', 'del', 'dello', 'dei', 
                      'degli', 'dell', 'degl', 'della', 'delle', 'in', 'nel', 'nello', 'nei', 'negli', 'nell', 'negl', 
                      'nella', 'nelle', 's', 'sul', 'sullo', 'sui', 'sugli', 'sull', 'sugl', 'sulla', 'sulle', 'per', 
                      'tra', 'contro', 'io', 't', 'lui', 'lei', 'noi', 'voi', 'loro', 'mio', 'mia', 'miei', 'mie', 
                      'tuo', 'tua', 'tuoi', 'tue', 'suo', 'sua', 'suoi', 'sue', 'nostro', 'nostra', 'nostri', 'nostre',
                      'vostro', 'vostra', 'vostri', 'vostre', 'mi', 'ti', 'ci', 'vi', 'lo', 'la', 'li', 'le', 'gli', 'ne', 
                      'il', 'un', 'uno', 'una', 'ma', 'ed', 'se', 'perché', 'anche', 'come', 'dov', 'dove', 'che', 'chi', 
                      'cui', 'non', 'più', 'quale', 'quanto', 'quanti', 'quanta', 'quante', 'quello', 'quelli', 'quella', 
                      'quelle', 'questo', 'questi', 'questa', 'queste', 'si', 'tutto', 'tutti', 'a', 'c', 'e', 'i', 'l', 
                      'o', 'ho', 'hai', 'ha', 'abbiamo', 'avete', 'hanno', 'abbia', 'abbiate', 'abbiano', 'avrà', 'avrai',
                      'avrò', 'avremo', 'avrete', 'avranno', 'avrei', 'avresti', 'avrebbe', 'avremmo', 'avreste', 'avrebbero',
                      'avevo', 'avevi', 'aveva', 'avevamo', 'avevate', 'avevano', 'ebbi', 'avesti', 'ebbe', 'avemmo', 'aveste',
                      'ebbero', 'avessi', 'avesse', 'avessimo', 'avessero', 'avendo', 'avuto', 'avuta', 'avuti', 'avute', 'sono', 
                      'sei', 'è', 'siamo', 'siete', 'sia', 'siate', 'siano', 'sarà', 'sarai', 'sarò', 'saremo', 'sarete', 
                      'saranno', 'sarei', 'saresti', 'sarebbe', 'saremmo', 'sareste', 'sarebbero', 'ero', 'eri', 'era', 
                      'eravamo', 'eravate', 'erano', 'fui', 'fosti', 'fu', 'fummo', 'foste', 'furono', 'fossi', 'fosse', 
                      'fossimo', 'fossero', 'essendo', 'faccio', 'fai', 'facciamo', 'fanno', 'faccia', 'facciate', 'facciano', 
                      'farà', 'farai', 'farò', 'faremo', 'farete', 'faranno', 'farei', 'faresti', 'farebbe', 'faremmo', 'fareste', 
                      'farebbero', 'facevo', 'facevi', 'faceva', 'facevamo', 'facevate', 'facevano', 'feci', 'facesti', 'fece', 
                      'facemmo', 'faceste', 'fecero', 'facessi', 'facesse', 'facessimo', 'facessero', 'facendo', 'sto', 'stai', 
                      'sta', 'stiamo', 'stanno', 'stia', 'stiate', 'stiano', 'starà', 'starai', 'starò', 'staremo', 'starete',
                      'staranno', 'starei', 'staresti', 'starebbe', 'staremmo', 'stareste', 'starebbero', 'stavo', 'stavi', 'stava', 
                      'stavamo', 'stavate', 'stavano', 'stetti', 'stesti', 'stette', 'stemmo', 'steste', 'stettero', 'stessi', 'stesse', 
                      'stessimo', 'stessero', 'stando', "e'"]

data = sc.textFile(path).zipWithIndex().map(lambda (words,idd): Row(idd= idd, words = words.split(" ")))
docDF = spark.createDataFrame(data)

remover = StopWordsRemover(inputCol="words", outputCol="clean_words", stopWords = italian_stop_words)
docDF = remover.transform(docDF)

Vector = CountVectorizer(inputCol="clean_words", outputCol="vectors")
model = Vector.fit(docDF)
result = model.transform(docDF)

corpus = result.select("idd", "vectors").rdd.map(lambda (x,y): [x,Vectors.fromML(y)]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3,maxIterations=100,optimizer='online')
topics = ldaModel.topicsMatrix()
vocabArray = model.vocabulary

weight = ldaModel.describeTopics(maxTermsPerTopic = 10)

wordNumbers = 10  # number of words per topic
topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))

def topic_render(topic):  # specify vector id of words to actual words
    terms = topic[0]
    result = []
    for i in range(wordNumbers):
        term = vocabArray[terms[i]]
        result.append(term)
    return result

topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()

#print(vocabArray(int(weight[0][0][0])))
i = 0 
for topic in range(len(topics_final)):
    print ("Topic" + str(topic) + ":")
    j = 0
    for term in topics_final[topic]:
        print (term + '  ' + str(weight[i][1][j]))
        j += 1
    i += 1
    print ('\n')

Topic0:
vigili  0.00713896446364
due  0.00708285329651
riesce  0.00608226977368
vigile  0.00602941855362
arrestato  0.00600086233067
pistola  0.00595320275603
stato  0.00592130229032
strada  0.00588526584937
mountainbike  0.00584885973868
Firenze,  0.00580306485212


Topic1:
strada  0.00650637475481
zona  0.0062763044834
omicidio.  0.00605221657151
alcuni  0.00599252848737
senza  0.00592982148909
finiti  0.00586466396878
accorge  0.00584910359443
dell'agente  0.00584843074309
uscire  0.0057550893171
Roberto  0.00574618949919


Topic2:
consentito  0.00670203116557
accaduto  0.00657406005837
dimora  0.00619547189446
soggiorno,  0.00606321292384
arrivano  0.00600156500634
magro.  0.00594173633438
municipale  0.00591620229118
agente  0.00590408111088
secondo  0.00590025595787
ragazzo  0.00579895070113


